In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import anndata 
import os
import mudata
import quiche as qu
from sketchKH import sketch
from supplementary_plot_helpers import *
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
sns.set_style('ticks')
plt.rcParams.update({
    "font.size": 10,
    "axes.titlesize": 12,
    "axes.labelsize": 10,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
})
%reload_ext autoreload
%load_ext autoreload
%autoreload 2
%matplotlib inline

### validation figures

In [ ]:
save_directory = os.path.join('publications', 'supplementary_figures', 'supplementary_figure18')
qu.pp.make_directory(save_directory)

In [ ]:
mdata_spain = mudata.read_h5mu(os.path.join('data', 'tnbc_spain', 'mdata', 'mdata_spain_study_corrected.h5ad'))
mdata_spain['quiche'].var[['SpatialFDR', 'logFC']] = mdata_spain['quiche'].var[['SpatialFDR', 'logFC']].astype('float')
scores_df_spain = pd.DataFrame(mdata_spain['quiche'].var.groupby('quiche_niche')['SpatialFDR'].median())
scores_df_spain.columns = ['pval']
scores_df_spain['logFC'] = mdata_spain['quiche'].var.groupby('quiche_niche')['logFC'].mean()
scores_df_spain = scores_df_spain[scores_df_spain['pval'] < 0.05]
ids = list(set(scores_df_spain.index).intersection(set(list(mdata_spain['quiche'].var['quiche_niche'].value_counts()[mdata_spain['quiche'].var['quiche_niche'].value_counts() >= 5].index))))
scores_df_spain = scores_df_spain.loc[ids]
scores_df_spain = scores_df_spain[(scores_df_spain.logFC > 1) | (scores_df_spain.logFC < -1)]
niches_spain = list(scores_df_spain.index)

In [ ]:
mdata_stanford = mudata.read_h5mu(os.path.join('data', 'tnbc_stanford', 'mdata', 'mdata_stanford.h5ad'))
mdata_stanford['quiche'].var[['SpatialFDR', 'logFC']] = mdata_stanford['quiche'].var[['SpatialFDR', 'logFC']].astype('float')
scores_df_stanford = pd.DataFrame(mdata_stanford['quiche'].var.groupby('quiche_niche')['SpatialFDR'].median())
scores_df_stanford.columns = ['pval']
scores_df_stanford['logFC'] = mdata_stanford['quiche'].var.groupby('quiche_niche')['logFC'].mean()
scores_df_stanford = scores_df_stanford[scores_df_stanford['pval'] < 0.05]
ids = list(set(scores_df_stanford.index).intersection(set(list(mdata_stanford['quiche'].var['quiche_niche'].value_counts()[mdata_stanford['quiche'].var['quiche_niche'].value_counts() >= 5].index))))
scores_df_stanford = scores_df_stanford.loc[ids]
scores_df_stanford = scores_df_stanford[(scores_df_stanford.logFC > 1) | (scores_df_stanford.logFC < -1)]
niches_stanford = list(scores_df_stanford.index)

In [ ]:
mdata_nt = mudata.read_h5mu(os.path.join('data', 'tnbc_ntpublic', 'mdata', 'mdata_ntpublic.h5ad'))
mdata_nt['quiche'].var[['SpatialFDR', 'logFC']] = mdata_nt['quiche'].var[['SpatialFDR', 'logFC']].astype('float')
scores_df_nt = pd.DataFrame(mdata_nt['quiche'].var.groupby('quiche_niche')['SpatialFDR'].median())
scores_df_nt.columns = ['pval']
scores_df_nt['logFC'] = mdata_nt['quiche'].var.groupby('quiche_niche')['logFC'].mean()
scores_df_nt = scores_df_nt[scores_df_nt['pval'] < 0.05]
ids = list(set(scores_df_nt.index).intersection(set(list(mdata_nt['quiche'].var['quiche_niche'].value_counts()[mdata_nt['quiche'].var['quiche_niche'].value_counts() >= 5].index))))
scores_df_nt = scores_df_nt.loc[ids]
scores_df_nt = scores_df_nt[(scores_df_nt.logFC > 1) | (scores_df_nt.logFC < -1)]
niches_nt = list(scores_df_nt.index)

## Supplementary Figure 18a

In [ ]:
plot_correlation(mdata_spain, save_directory, 'supplementary_figure18a_spain')
plot_correlation(mdata_stanford, save_directory, 'supplementary_figure18a_stanford')
plot_correlation(mdata_nt, save_directory, 'supplementary_figure18a_nt')

## Supplementary Figure 18b

In [ ]:
plot_nhood_distances(mdata_spain, niches_spain, save_directory, 'supplementary_figure18b_spain')
plot_nhood_distances(mdata_stanford, niches_stanford, save_directory, 'supplementary_figure18b_stanford')
plot_nhood_distances(mdata_nt, niches_nt, save_directory, 'supplementary_figure18b_nt')

In [ ]:
# plot_prop(mdata_spain, niches_spain, save_directory, 'niche_labels_spain')
# plot_prop(mdata_stanford, niches_stanford, save_directory, 'niche_labels_stanford')
# plot_prop(mdata_nt,niches_nt, save_directory, 'niche_labels_nt')

## Supplementary Figure 18c

In [ ]:
mdata_spain = mudata.read_h5mu(os.path.join('data', 'tnbc_spain','mdata', 'mdata_spain_study_corrected.h5ad'))
mdata_spain['quiche'].var[['logFC', 'SpatialFDR', 'PValue']] = mdata_spain['quiche'].var[['logFC', 'SpatialFDR', 'PValue']].astype('float')

scores_df_spain = pd.DataFrame(mdata_spain['quiche'].var.groupby('quiche_niche')['SpatialFDR'].median())
scores_df_spain.columns = ['pval']
scores_df_spain['logFC'] = mdata_spain['quiche'].var.groupby('quiche_niche')['logFC'].mean()
scores_df_spain = scores_df_spain[scores_df_spain['pval'] < 0.05]
ids = list(set(scores_df_spain.index).intersection(set(list(mdata_spain['quiche'].var['quiche_niche'].value_counts()[mdata_spain['quiche'].var['quiche_niche'].value_counts() >= 5].index))))
scores_df_spain = scores_df_spain.loc[ids]
scores_df_spain = scores_df_spain[(scores_df_spain.logFC > 0.5) | (scores_df_spain.logFC < -0.5)]
niches_spain = list(scores_df_spain.index)

cov_count_df = qu.tl.compute_patient_proportion(mdata_spain,
                                niches = niches_spain,
                                feature_key = 'quiche',
                                annot_key = 'quiche_niche',
                                patient_key = 'Patient_ID',
                                design_key = 'Relapse',
                                patient_niche_threshold = 5)
cov_count_df_frequent = cov_count_df[cov_count_df['patient_count'] >= 3]

In [ ]:
predicted_niches = list(cov_count_df_frequent.quiche_niche)

In [ ]:
design = '~Relapseshuffled'
model_contrasts = 'Relapseshuffled1-Relapseshuffled0'
test_key = 'Patient_ID'
df_permute = []
for i in range(0, 20):
    df_relapse = mdata_spain['spatial_nhood'].obs[['Patient_ID', 'Relapse']].drop_duplicates().copy() ##maintain link between patient and relapse
    df_relapse['Relapseshuffled'] = np.random.permutation(df_relapse['Relapse']) 
    df_relapse.drop(columns = 'Relapse', inplace=True)
    df_merge = pd.merge(mdata_spain['spatial_nhood'].obs, df_relapse, on = 'Patient_ID')
    df_merge.index =  mdata_spain['spatial_nhood'].obs_names
    mdata_spain['spatial_nhood'].obs = df_merge
    mdata_spain_permute = qu.tl.quicheDA(mdata_spain['spatial_nhood'], design = design, model_contrasts=model_contrasts, patient_key = test_key)
    annotations = qu.tl.label_niches(mdata_spain, nlargest = 3, min_perc = 0.1)
    try:
        mdata_spain_permute['milo'].var['quiche_niche'] = annotations.values
    except:
        mdata_spain_permute['milo'].var['quiche_niche'] = annotations
    
    df_permute_ = mdata_spain_permute['milo'].var[np.isin(mdata_spain_permute['milo'].var['quiche_niche'], predicted_niches)].groupby('quiche_niche')[['logFC', 'SpatialFDR']].mean()
    df_permute_['trial'] = i
    df_permute_ = df_permute_.melt(id_vars= ['trial'], ignore_index=False)
    df_permute.append(df_permute_)
    del mdata_spain['spatial_nhood'].obs['Relapseshuffled']

df_permute_concat  = pd.concat(df_permute)
df_permute_concat.to_csv(os.path.join('data','output_files', 'spain_shuffled.csv'))

In [ ]:
fig = plt.figure(figsize=(6, 10))
gs = GridSpec(1, 2, width_ratios=[1, 1], wspace=0.1)

ax0 = plt.subplot(gs[0])
sns.boxplot(
    data=df_permute_concat[df_permute_concat['variable'] == 'logFC'],
    y=df_permute_concat[df_permute_concat['variable'] == 'logFC'].index,
    x="value",
    order=predicted_niches,
    orient="h",
    color="gray",
    ax=ax0,
    width=0.6,
    fliersize=0.8
)
ax0.axvline(x=0, linestyle='--', color='black', linewidth=1)
ax0.set_xlabel("Log2(fold change)", fontsize = 10)
ax0.set_ylabel('QUICHE niche neighborhoods')
ax1 = plt.subplot(gs[1], sharey=ax0)
sns.boxplot(
    data=df_permute_concat[df_permute_concat['variable'] == 'SpatialFDR'],
    y=df_permute_concat[df_permute_concat['variable'] == 'SpatialFDR'].index,
    x="value",
    order=predicted_niches,
    orient="h",
    color="gray",
    ax=ax1,
    width=0.6,
    fliersize=0.8
)
ax1.axvline(x=0.05, linestyle='--', color='black', linewidth=1, label = 'FDR < 0.05')
ax1.set_xlabel("SpatialFDR", fontsize = 10)
ax1.set_ylabel('')
plt.legend()
plt.setp(ax1.get_yticklabels(), visible=False)
plt.tight_layout()
plt.savefig(os.path.join(save_directory, 'supplementary_figure18c_spain.pdf'), bbox_inches = 'tight')


In [ ]:
mdata_stanford = mudata.read_h5mu(os.path.join('data', 'tnbc_stanford', 'mdata', 'mdata_stanford.h5ad'))
mdata_stanford['quiche'].var[['SpatialFDR', 'logFC']] = mdata_stanford['quiche'].var[['SpatialFDR', 'logFC']].astype('float')
scores_df_stanford = pd.DataFrame(mdata_stanford['quiche'].var.groupby('quiche_niche')['SpatialFDR'].median())
scores_df_stanford.columns = ['pval']
scores_df_stanford['logFC'] = mdata_stanford['quiche'].var.groupby('quiche_niche')['logFC'].mean()
scores_df_stanford = scores_df_stanford[scores_df_stanford['pval'] < 0.05]
ids = list(set(scores_df_stanford.index).intersection(set(list(mdata_stanford['quiche'].var['quiche_niche'].value_counts()[mdata_stanford['quiche'].var['quiche_niche'].value_counts() >= 5].index))))
scores_df_stanford = scores_df_stanford.loc[ids]
scores_df_stanford = scores_df_stanford[(scores_df_stanford.logFC > 1) | (scores_df_stanford.logFC < -1)]
niches_stanford = list(scores_df_stanford.index)

In [ ]:
mdata_stanford['quiche'].var['SpatialFDR'] = mdata_stanford['quiche'].var['SpatialFDR'].astype('float')
mdata_stanford['quiche'].var['logFC'] = mdata_stanford['quiche'].var['logFC'].astype('float')
scores_df_stanford = pd.DataFrame(mdata_stanford['quiche'].var.groupby('quiche_niche')['SpatialFDR'].median())
scores_df_stanford.columns = ['pval']
scores_df_stanford['logFC'] = mdata_stanford['quiche'].var.groupby('quiche_niche')['logFC'].mean()
scores_df_stanford = scores_df_stanford[scores_df_stanford['pval'] < 0.05]
ids = list(set(scores_df_stanford.index).intersection(set(list(mdata_stanford['quiche'].var['quiche_niche'].value_counts()[mdata_stanford['quiche'].var['quiche_niche'].value_counts() >= 5].index))))
scores_df_stanford = scores_df_stanford.loc[ids]
scores_df_stanford = scores_df_stanford[(scores_df_stanford.logFC > 0.5) | (scores_df_stanford.logFC < -0.5)]
niches_stanford = list(scores_df_stanford.index)

cov_count_df = qu.tl.compute_patient_proportion(mdata_stanford,
                                niches = niches_stanford,
                                feature_key = 'quiche',
                                annot_key = 'quiche_niche',
                                patient_key = 'Patient_ID',
                                design_key = 'RECURRENCE_LABEL',
                                patient_niche_threshold = 5)

cov_count_df_frequent = cov_count_df[cov_count_df['patient_count'] >= 3]

In [ ]:
predicted_niches = list(cov_count_df_frequent.quiche_niche)

In [ ]:
design = '~RECURRENCE_LABELshuffled'
model_contrasts = 'RECURRENCE_LABELshuffledPOSITIVE-RECURRENCE_LABELshuffledNEGATIVE'
test_key = 'Patient_ID'
df_permute = []
for i in range(0, 20):
    print(i)
    df_relapse = mdata_stanford['spatial_nhood'].obs[['Patient_ID', 'RECURRENCE_LABEL']].drop_duplicates().copy() ##maintain link between patient and relapse
    df_relapse['RECURRENCE_LABELshuffled'] = np.random.permutation(df_relapse['RECURRENCE_LABEL'])
    df_relapse.drop(columns = 'RECURRENCE_LABEL', inplace=True)
    df_merge = pd.merge(mdata_stanford['spatial_nhood'].obs, df_relapse, on = 'Patient_ID')
    df_merge.index =  mdata_stanford['spatial_nhood'].obs_names
    mdata_stanford['spatial_nhood'].obs = df_merge
    mdata_permute = qu.tl.quicheDA(mdata_stanford['spatial_nhood'], design = design, model_contrasts=model_contrasts, patient_key = test_key)
    annotations = qu.tl.label_niches(mdata_stanford, nlargest = 3, min_perc = 0.1)
    try:
        mdata_permute['milo'].var['quiche_niche'] = annotations.values
    except:
        mdata_permute['milo'].var['quiche_niche'] = annotations
    
    df_permute_ = mdata_permute['milo'].var[np.isin(mdata_permute['milo'].var['quiche_niche'], predicted_niches)].groupby('quiche_niche')[['logFC', 'SpatialFDR']].mean()
    df_permute_['trial'] = i
    df_permute_ = df_permute_.melt(id_vars= ['trial'], ignore_index=False)
    df_permute.append(df_permute_)
    del mdata_stanford['spatial_nhood'].obs['RECURRENCE_LABELshuffled']
df_permute_concat  = pd.concat(df_permute)
df_permute_concat.to_csv(os.path.join('data','output_files','stanford_shuffled.csv'))

In [ ]:
fig = plt.figure(figsize=(6, 5.5))
gs = GridSpec(1, 2, width_ratios=[1, 1], wspace=0.1)
ax0 = plt.subplot(gs[0])
sns.boxplot(
    data=df_permute_concat[df_permute_concat['variable'] == 'logFC'],
    y=df_permute_concat[df_permute_concat['variable'] == 'logFC'].index,
    x="value",
    order=predicted_niches,
    orient="h",
    color="gray",
    ax=ax0,
    width=0.6,
    fliersize=0.8
)
ax0.axvline(x=0, linestyle='--', color='black', linewidth=1)
ax0.set_xlabel("Log2(fold change)", fontsize = 10)
ax0.set_ylabel('QUICHE niche neighborhoods')
ax1 = plt.subplot(gs[1], sharey=ax0)
sns.boxplot(
    data=df_permute_concat[df_permute_concat['variable'] == 'SpatialFDR'],
    y=df_permute_concat[df_permute_concat['variable'] == 'SpatialFDR'].index,
    x="value",
    order=predicted_niches,
    orient="h",
    color="gray",
    ax=ax1,
    width=0.6,
    fliersize=0.8
)
ax1.axvline(x=0.05, linestyle='--', color='black', linewidth=1, label = 'FDR < 0.05')
ax1.set_xlabel("SpatialFDR", fontsize = 10)
ax1.set_ylabel('')
plt.legend()
plt.setp(ax1.get_yticklabels(), visible=False)
plt.tight_layout()
plt.savefig(os.path.join(save_directory, 'supplementary_figure18c_stanford.pdf'), bbox_inches = 'tight')


In [ ]:
mdata = mudata.read_h5mu(os.path.join('data', 'tnbc_ntpublic', 'mdata', 'mdata_ntpublic.h5ad'))
mdata['quiche'].var['SpatialFDR'] = mdata['quiche'].var['SpatialFDR'].astype('float')
mdata['quiche'].var['logFC'] = mdata['quiche'].var['logFC'].astype('float')
scores_df_nt = pd.DataFrame(mdata['quiche'].var.groupby('quiche_niche')['SpatialFDR'].median())
scores_df_nt.columns = ['pval']
scores_df_nt['logFC'] = mdata['quiche'].var.groupby('quiche_niche')['logFC'].mean()
scores_df_nt = scores_df_nt[scores_df_nt['pval'] < 0.05]
ids = list(set(scores_df_nt.index).intersection(set(list(mdata['quiche'].var['quiche_niche'].value_counts()[mdata['quiche'].var['quiche_niche'].value_counts() >= 5].index))))
scores_df_nt = scores_df_nt.loc[ids]
scores_df_nt = scores_df_nt[(scores_df_nt.logFC > 0.5) | (scores_df_nt.logFC < -0.5)]
niches_nt = list(scores_df_nt.index)

cov_count_df = qu.tl.compute_patient_proportion(mdata,
                                niches = niches_nt,
                                feature_key = 'quiche',
                                annot_key = 'quiche_niche',
                                patient_key = 'Patient_ID',
                                design_key = 'pCR',
                                patient_niche_threshold = 5)
cov_count_df_frequent = cov_count_df[cov_count_df['patient_count'] >= 2]

In [ ]:
predicted_niches = list(cov_count_df_frequent.quiche_niche)

In [ ]:
design = '~pCRshuffled'
model_contrasts = 'pCRshuffledRD-pCRshuffledpCR'
test_key = 'Patient_ID'
df_permute = []
for i in range(0, 20):
    print(i)
    df_relapse = mdata['spatial_nhood'].obs[['Patient_ID', 'pCR']].drop_duplicates().copy() ##maintain link between patient and relapse
    df_relapse['pCRshuffled'] = np.random.permutation(df_relapse['pCR'])
    df_relapse.drop(columns = 'pCR', inplace=True)
    df_merge = pd.merge(mdata['spatial_nhood'].obs, df_relapse, on = 'Patient_ID')
    df_merge.index =  mdata['spatial_nhood'].obs_names
    mdata['spatial_nhood'].obs = df_merge
    mdata_permute = qu.tl.quicheDA(mdata['spatial_nhood'], design = design, model_contrasts=model_contrasts, patient_key = test_key)
    annotations = qu.tl.label_niches(mdata, nlargest = 3, min_perc = 0.1)
    try:
        mdata_permute['milo'].var['quiche_niche'] = annotations.values
    except:
        mdata_permute['milo'].var['quiche_niche'] = annotations
    
    df_permute_ = mdata_permute['milo'].var[np.isin(mdata_permute['milo'].var['quiche_niche'], predicted_niches)].groupby('quiche_niche')[['logFC', 'SpatialFDR']].mean()
    df_permute_['trial'] = i
    df_permute_ = df_permute_.melt(id_vars= ['trial'], ignore_index=False)
    df_permute.append(df_permute_)
    del mdata['spatial_nhood'].obs['pCRshuffled']
df_permute_concat  = pd.concat(df_permute)
df_permute_concat.to_csv(os.path.join('data','output_files','neotrip_shuffled.csv'))

In [ ]:
fig = plt.figure(figsize=(6, 5))
gs = GridSpec(1, 2, width_ratios=[1, 1], wspace=0.1)
ax0 = plt.subplot(gs[0])
sns.boxplot(
    data=df_permute_concat[df_permute_concat['variable'] == 'logFC'],
    y=df_permute_concat[df_permute_concat['variable'] == 'logFC'].index,
    x="value",
    order=predicted_niches,
    orient="h",
    color="gray",
    ax=ax0,
    width=0.6,
    fliersize=0.8
)
ax0.axvline(x=0, linestyle='--', color='black', linewidth=1)
ax0.set_xlabel("Log2(fold change)", fontsize = 10)
ax0.set_ylabel('QUICHE niche neighborhoods')
ax1 = plt.subplot(gs[1], sharey=ax0)
sns.boxplot(
    data=df_permute_concat[df_permute_concat['variable'] == 'SpatialFDR'],
    y=df_permute_concat[df_permute_concat['variable'] == 'SpatialFDR'].index,
    x="value",
    order=predicted_niches,
    orient="h",
    color="gray",
    ax=ax1,
    width=0.6,
    fliersize=0.8
)
ax1.axvline(x=0.05, linestyle='--', color='black', linewidth=1, label = 'FDR < 0.05')  # Vertical dotted line at 0.05
ax1.set_xlabel("SpatialFDR", fontsize = 10)
ax1.set_ylabel('')
plt.legend()
plt.setp(ax1.get_yticklabels(), visible=False)
plt.tight_layout()
plt.savefig(os.path.join(save_directory, 'supplementary_figure18c_neotrip.pdf'), bbox_inches = 'tight')